In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
from sklearn.model_selection import train_test_split

In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize

In [ ]:
import hd_fusion

In [ ]:
SEED = 1234
device = 'mps' if torch.cuda.is_available() else 'cpu'
np.random.seed(SEED)
torch.manual_seed(SEED)

# AerialPhoto

In [9]:
images = torch.zeros(size=(1400, 3, 256, 256))
labels = torch.zeros(size=(1400, ))
resize = Resize(size=(256, 256))

idx = 0
path1s = os.listdir('EarthonCanvas/AerialPhoto')

for label, path1 in enumerate(path1s):
    path2s = os.listdir(f'EarthonCanvas/AerialPhoto/{path1}')
    
    for path2 in path2s:
        labels[idx] = label        
        image = Image.open(f'./EarthonCanvas/AerialPhoto/{path1}/{path2}')
        image = resize(image)
        image = ToTensor()(image)
        images[idx] = image
        idx = idx+1

In [10]:
x, x_test, y, y_test = train_test_split(images, labels, test_size=0.3, random_state=1234)

# Normalize
scaler = sklearn.preprocessing.Normalizer().fit(x.reshape(-1, 256*256*3))
X1 = scaler.transform(x.reshape(-1, 256*256*3))
X1_test = scaler.transform(x_test.reshape(-1, 256*256*3))

X1 = torch.from_numpy(X1).float()
X1_test = torch.from_numpy(X1_test).float()
Y1 = y.long()
Y1_test = y_test.long()

# Aerialsketch

In [13]:
images = torch.zeros(size=(1400, 3, 256, 256))
labels = torch.zeros(size=(1400, ))
resize = Resize(size=(256, 256))

idx = 0
path1s = os.listdir('EarthonCanvas/Aerialsketch')

for label, path1 in enumerate(path1s):
    path2s = os.listdir(f'EarthonCanvas/Aerialsketch/{path1}')
    
    for path2 in path2s:
        labels[idx] = label        
        image = Image.open(f'./EarthonCanvas/Aerialsketch/{path1}/{path2}')
        image = resize(image)
        image = ToTensor()(image)
        images[idx] = image
        idx = idx+1

In [14]:
x, x_test, y, y_test = train_test_split(images, labels, test_size=0.3, random_state=1234)

# Normalize
scaler = sklearn.preprocessing.Normalizer().fit(x.reshape(-1, 256*256*3))
X2 = scaler.transform(x.reshape(-1, 256*256*3))
X2_test = scaler.transform(x_test.reshape(-1, 256*256*3))

X2 = torch.from_numpy(X2).float()
X2_test = torch.from_numpy(X2_test).float()
Y2 = y.long()
Y2_test = y_test.long()

In [ ]:
classes_1 = Y1.unique().size(0)
classes_2 = Y2.unique().size(0)
if classes_1 != classes_2:
    print("ERROR")
else:
    classes = classes_1

features_1 = X1.size(1)
features_2 = X2.size(1)
dims = [1000, 3000, 6000, 10000]
alphas = range(0, 1, 0.1)
max_alphas = []
for dim in dims:
    max_alpha = -1
    max_acc = 0
    model = hd_fusion.HDFusion(classes, features_1, features_2, dim, device='mps')
    model.set_data(X1=X1, Y1=Y1, X2=X2, Y2=Y2)
    model.train()
    for a in alphas:
        model.train_fusion(alpha=a)
        acc, enc_X1_t, enc_X2_t = model.test_inference(X1=X1_test, Y1=Y1_test, X2=X2_test, Y2=Y2_test)
        model.filter_df(filename=None, X1=enc_X1_t, X2=enc_X2_t)
        if max_acc < acc:
            max_alpha = a
    max_alphas.append(max_alpha)
    print("For dim {} the best alpha is", max_alpha)


=======dim=1000========
Training...
training time : 7.154757022857666
acc :  0.949999988079071
acc_test :  0.3261904716491699
inference_time :  13.092463731765747
=======dim=4000========
Training...
training time : 15.9873948097229
acc :  0.9602040648460388
acc_test :  0.34761905670166016
inference_time :  31.914437770843506
=======dim=10000========
Training...
training time : 36.84968590736389
acc :  0.9591836929321289
acc_test :  0.3642857074737549
inference_time :  74.43924880027771


In [16]:
classes_1 = Y1.unique().size(0)
classes_2 = Y2.unique().size(0)
if classes_1 != classes_2:
    print("ERROR")
else:
    classes = classes_1

features_1 = X1.size(1)
features_2 = X2.size(1)
dims = [1000, 3000, 6000, 10000]
alphas = range(0, 1, 0.1)
for max_alpha, dim in zip(max_alphas, dims):
    model = hd_fusion.HDFusion(classes, features_1, features_2, dim, device='mps')
    model.set_data(X1=X1, Y1=Y1, X2=X2, Y2=Y2)
    model.train()
    model.train_fusion(alpha=max_alpha)
    acc, enc_X1_t, enc_X2_t = model.test_inference(X1=X1_test, Y1=Y1_test, X2=X2_test, Y2=Y2_test)
    model.filter_df(filename='./results/fixed_fusion_results_{}.csv'.format(dim), X1=enc_X1_t, X2=enc_X2_t)

Predicted,0,1,2,3,4,5,6,7,8,9,10,11,12,13,All
Actual,,,,,,,,,,,,,,,
0,1,10,0,0,0,1,3,1,0,0,6,4,0,0,26
1,0,33,0,0,0,0,2,0,0,0,0,0,0,0,35
2,2,7,3,0,0,5,7,1,0,0,7,2,0,2,36
3,0,0,0,10,5,1,1,2,1,2,1,2,8,1,34
4,0,0,0,7,7,0,0,0,4,2,0,3,6,3,32
5,1,13,0,1,0,1,4,0,0,0,5,1,1,4,31
6,0,7,0,0,0,1,12,0,0,1,8,1,0,0,30
7,1,7,0,0,0,1,2,2,0,0,3,4,0,0,20
8,0,0,0,3,3,0,1,0,5,8,3,1,4,1,29
